# 1. Prompt the LLM with only the question (without any context and with LGPD)

In [1]:
import pandas as pd
from getpass import getpass
from openai import OpenAI
from tqdm import tqdm
from groq import Groq
import time

In [15]:
OPENAI_KEY = getpass("KEY OpenAI")
DEEPSEEK_KEY = getpass("KEY DeepSeek")
GROQ_API = getpass("KEY Groq")
SABIA_API = getpass("KEY Maritaca")

KEY OpenAI ········
KEY DeepSeek ········
KEY Groq ········
KEY Maritaca ········


In [3]:
dataset_file = "dataset/gdpr-br-qa.jsonl"
gdpr_br_file = "lgpd/lgpd.txt"

In [4]:
# Abrir e ler o conteúdo do arquivo
with open(gdpr_br_file, "r", encoding="utf-8") as file:
    gdpr_br = file.read()

In [5]:
system_msg_without_context = """
Seu papel é resolver questões sobre a Lei Geral de Proteção de Dados Pessoais (LGPD) do Brasil. Para isso, você deverá obedecer às seguintes regras:

1. O usuário irá te passar uma questão que você deverá responder.
2. Essa questão pode ser de dois tipos. Pode ser de múltipla escolha, situação em que você deverá avaliar o enunciado e decidir por uma das alternativas apresentadas, ou pode ser de <CERTO> ou <ERRADO>, situação em que você deverá avaliar se o enunciado está correto ou não. O que diferencia as questões é a presença ou ausência de alternativas. Se tiver alternativas, é de múltipla escolha. Se não houver, é de <CERTO> ou <ERRADO>
3. A sua resposta deve ser direta, sem nenhuma marcação (pontuação, aspas ou qualquer outro caractere além da resposta).
3.1. Se for uma questão de múltiplica escolha, indique apenas a letra que responde ao enunciado.
3.2. Se for uma questão de <CERTO> ou <ERRADO> responda com "Certo" (para certo) ou "Errado" para errado.
""".strip()

In [6]:
system_msg_with_gdpr = """
Considere a Lei Geral de Proteção de Dados Pessoais (LGPD), transcrita abaixo:

[[INICIO LGPD]]
{gdpr}
[[FIM LGPD]]

Seu papel é resolver questões sobre a LGPD. Para isso, você deverá obedecer às seguintes regras:

1. O usuário irá te passar uma questão que você deverá responder.
2. Essa questão pode ser de dois tipos. Pode ser de múltipla escolha, situação em que você deverá avaliar o enunciado e decidir por uma das alternativas apresentadas, ou pode ser de <CERTO> ou <ERRADO>, situação em que você deverá avaliar se o enunciado está correto ou não. O que diferencia as questões é a presença ou ausência de alternativas. Se tiver alternativas, é de múltipla escolha. Se não houver, é de <CERTO> ou <ERRADO>
3. A sua resposta deve ser direta, sem nenhuma marcação (pontuação, aspas ou qualquer outro caractere além da resposta).
3.1. Se for uma questão de múltiplica escolha, indique apenas a letra que responde ao enunciado.
3.2. Se for uma questão de <CERTO> ou <ERRADO> responda com "Certo" (para certo) ou "Errado" para errado.
""".format(gdpr=gdpr_br).strip()

In [16]:
# Parâmetros dos modelos

NO_CTX_GPT_4o_MINI = {
    "EXPERIMENT_NAME": "no_context_gpt-4o-mini-2024-07-18",
    "CLIENT": OpenAI(api_key=OPENAI_KEY, base_url=None),
    "MODEL": "gpt-4o-mini-2024-07-18",
    "SYSTEM_MSG": system_msg_without_context
}

NO_CTX_DEEPSEEK_CHAT = {
    "EXPERIMENT_NAME": "no_context_deepseek-chat",
    "CLIENT": OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com"),
    "MODEL": "deepseek-chat",
    "SYSTEM_MSG": system_msg_without_context
}

NO_CTX_LLAMA3_3 = {
    "EXPERIMENT_NAME": "no_context_llama-3.3-70b-versatile",
    "CLIENT": Groq(api_key=GROQ_API),
    "MODEL": "llama-3.3-70b-versatile",
    "SYSTEM_MSG": system_msg_without_context
}

NO_CTX_SABIA3 = {
    "EXPERIMENT_NAME": "no_context_sabia-3-2024-12-11",
    "CLIENT": OpenAI(api_key=SABIA_API, base_url="https://chat.maritaca.ai/api"),
    "MODEL": "sabia-3-2024-12-11",
    "SYSTEM_MSG": system_msg_without_context
}

CTX_GPT_4o_MINI = {
    "EXPERIMENT_NAME": "with_gdpr_gpt-4o-mini-2024-07-18",
    "CLIENT": OpenAI(api_key=OPENAI_KEY, base_url=None),
    "MODEL": "gpt-4o-mini-2024-07-18",
    "SYSTEM_MSG": system_msg_with_gdpr
}

CTX_DEEPSEEK_CHAT = {
    "EXPERIMENT_NAME": "with_gdpr_deepseek-chat",
    "CLIENT": OpenAI(api_key=DEEPSEEK_KEY, base_url="https://api.deepseek.com"),
    "MODEL": "deepseek-chat",
    "SYSTEM_MSG": system_msg_with_gdpr
}

CTX_LLAMA3_3 = {
    "EXPERIMENT_NAME": "with_gdpr_llama-3.3-70b-versatile",
    "CLIENT": Groq(api_key=GROQ_API),
    "MODEL": "llama-3.3-70b-versatile",
    "SYSTEM_MSG": system_msg_with_gdpr
}

CTX_SABIA3 = {
    "EXPERIMENT_NAME": "with_gdpr_sabia-3-2024-12-11",
    "CLIENT": OpenAI(api_key=SABIA_API, base_url="https://chat.maritaca.ai/api"),
    "MODEL": "sabia-3-2024-12-11",
    "SYSTEM_MSG": system_msg_with_gdpr
}

EXPERIMENTS = [NO_CTX_GPT_4o_MINI, NO_CTX_LLAMA3_3, NO_CTX_DEEPSEEK_CHAT, NO_CTX_SABIA3,
               CTX_GPT_4o_MINI, CTX_LLAMA3_3, CTX_DEEPSEEK_CHAT, CTX_SABIA3]

In [8]:
df_dataset = pd.read_json(dataset_file, lines=True, encoding='utf-8')

In [9]:
# Create columns to represent the experiment results
for experiment in EXPERIMENTS:
    experiment_name = experiment['EXPERIMENT_NAME']
    llm_model = experiment['MODEL']

    # Create columns for the experiment results
    if experiment_name not in df_dataset.columns:
        df_dataset[experiment_name] = None

In [10]:
def get_answer_llm(experiment, formatted_question):
    llm_model = experiment['MODEL']
    client = experiment['CLIENT']
    response = client.chat.completions.create(
        model=llm_model,
        messages=[
            {"role": "system", "content": experiment['SYSTEM_MSG']},
            {"role": "user", "content": formatted_question},
        ],
        stream=False
    )
    content = response.choices[0].message.content
    
    return content

In [32]:
for experiment in EXPERIMENTS:
    experiment_name = experiment['EXPERIMENT_NAME']
    llm_model = experiment['MODEL']
    
    print(f"Experiment: {experiment_name}")
       
    for index, row in tqdm(df_dataset.iterrows(), total=len(df_dataset)):           
        # Run only if we don't have the results
        if pd.isna(row[experiment_name]) or row[experiment_name] == '':
            df_dataset.at[index, experiment_name] = get_answer_llm(experiment, row['Formatted Question'])
            
            # Update the dataset with the results
            with open(dataset_file, 'w', encoding='utf-8') as f:
                df_dataset.to_json(f, orient='records', lines=True, force_ascii=False)

Experiment: no_context_gpt-4o-mini-2024-07-18


100%|██████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 9375.53it/s]


Experiment: no_context_llama-3.3-70b-versatile


100%|█████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 16355.89it/s]


Experiment: no_context_deepseek-chat


...

100%|████████████████████████████████████████████████████████████████████████████████| 700/700 [33:03<00:00,  2.83s/it]


Experiment: with_gdpr_deepseek-chat


100%|█████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 84244.72it/s]


Experiment: with_gdpr_sabia-3-2024-12-11


100%|█████████████████████████████████████████████████████████████████████████████| 700/700 [00:00<00:00, 52253.38it/s]


In [33]:
# Show results for each experiment
for experiment in EXPERIMENTS:
    experiment_name = experiment['EXPERIMENT_NAME']
    avg = (df_dataset['Answer'] == df_dataset[experiment_name]).mean()
    print(f"Experiment: {experiment_name}. Avg: {avg}.")

Experiment: no_context_gpt-4o-mini-2024-07-18. Avg: 0.7571428571428571.
Experiment: no_context_llama-3.3-70b-versatile. Avg: 0.7314285714285714.
Experiment: no_context_deepseek-chat. Avg: 0.85.
Experiment: no_context_sabia-3-2024-12-11. Avg: 0.8671428571428571.
Experiment: with_gdpr_gpt-4o-mini-2024-07-18. Avg: 0.8328571428571429.
Experiment: with_gdpr_llama-3.3-70b-versatile. Avg: 0.27714285714285714.
Experiment: with_gdpr_deepseek-chat. Avg: 0.9414285714285714.
Experiment: with_gdpr_sabia-3-2024-12-11. Avg: 0.78.
